<div class="alert alert-block alert-info">
    <img src='iberostar.png' width="200" height="200"/>
    <h1><center>Iberostar Hotel Cancún</center></h1>  
    <p><center>Set competitivo de FACC</center></p>  
</div>

<div class="alert alert-block alert-warning">
    <h3><center>Importaciones</center></h3>
</div>

In [13]:
###
# Importadción de librerias del sistema
###
import os
import time
import datetime
from datetime import date
import random
import csv
print("Librerias del sistema listas")
###
# Importación de librerias para selenium
###
try:
    from selenium import webdriver
    from selenium.webdriver.common.keys import Keys
    from selenium.webdriver.common.by import By
    print("Librerias para selenium listas")
except():
    print("Instalando selenium en el enviroment")
    import sys
    !{sys.executable} -m pip install -U selenium
###
# Importación librerias urllib y BautifulSoup
###
try:
    import urllib
    from bs4 import BeautifulSoup
    print("Librerias listas de urllib y BeautifulSoup listas")
except():
    print("Instalando BeautifulSoup en el enviroment")
    import sys
    !{sys.executable} -m pip install -U BeautifulSoup
###
# Extracción del precio del dolar hoy
###
try:
    from forex_python.converter import CurrencyRates
except():
    print("Instalando la libreria CurrencyrRates")
    import sys
    !{sys.executable} -m pip install -U forex-python
###
# Importación de librerias para dataframe
###
import pandas as pd
print("Librerias para pandas listas")

Librerias del sistema listas
Librerias para selenium listas
Librerias listas de urllib y BeautifulSoup listas
Librerias para pandas listas


<div class="alert alert-block alert-warning">
    <h3><center>Métodos</center></h3>
</div>

In [14]:
def _connect_iberostar():
    """Método utilizado para conectarte a la url del hotel HRHC
       Parametros: <None>
       Retorno: <selenium.webdriver> tipo: selenium.webdriver --> Es el bot ya levantado y configurado"""
    #url = "https://booking.iberostar.com/book/Availability.aspx?codiconc=160&conccodi=76&cp_tealium=&edadpersona0_0=30&edadpersona0_1=30&fechafin="+str(date2.day)+"%2F"+str(date2.month)+"%2F"+str(date2.year)+"&fechaini="+str(date1.day)+"%2F"+str(date1.month)+"%2F"+str(date1.year)+"&idiocodi=1&monecodi=MXN&numerohabitaciones=1&numeropersonas0=2&origen_soporte=IBE&search_origin=hotel"
    chromeOptions = webdriver.ChromeOptions()
    chromeOptions.add_argument("--incognito")
    driver = webdriver.Chrome("chromedriver.exe",options=chromeOptions)
    return driver

def _update_page(driver,date1,date2): #card b-room-card
    """Método utilizado para actualizar la página web de la consulta, ya que en iberostar no podemos manipular los botones desde
       su página de inicio, si podemos hacer que al mover los parámetros de la url podamos seguir iterando en el.
       Parámetros: <selenium.webdriver> tipo: selenium.webdriver --> Es el bot ya levantado y configurado
       Retorno: <None>"""
    driver.get("https://booking.iberostar.com/book/Availability.aspx?codiconc=160&conccodi=76&cp_tealium=&edadpersona0_0=30&edadpersona0_1=30&fechafin="+str(date2.day)+"%2F"+str(date2.month)+"%2F"+str(date2.year)+"&fechaini="+str(date1.day)+"%2F"+str(date1.month)+"%2F"+str(date1.year)+"&idiocodi=1&monecodi=MXN&numerohabitaciones=1&numeropersonas0=2&origen_soporte=IBE&search_origin=hotel")
    _wait(5)
    return None

def _get_price(driver,dolar):
    """Método utilizado para obtener la tarifa y el nombre de la habitación.
       Párametros: <selenium.webdriver> tipo: selenium.webdriver --> Es el bot ya levantado y configurado
       Retorno: <lista_nombre,lista_habitacion> tipo: list --> Son las listas que contendran el nombre de la habitación
                así como su tarifa en dólares"""
    lista_nombre = []
    l_aux = []
    lista_precio_usd = []
    lista_precio_mxn = []
    str_nombre = 'a[class="link-details"]'
    str_habitacion = 'span[class="new-price"]'
    try:
        objs_titles = driver.find_elements_by_css_selector(str_nombre)
        l_aux = list(map(lambda x:x.text if x.text!="Ver más detalles" else "-",objs_titles))
    except:
        print("Elmento titulos no encontrado")
        l_aux = ["-"]
    for element in l_aux:
        if element != "-":
            lista_nombre.append(element) 
    #print(lista_nombre)
    try: 
        objs_tarifa = driver.find_elements_by_css_selector(str_habitacion)
        for element in objs_tarifa: 
            if element.text.split(" ")[0] == "USD" and element.text != "":
                if "," in element.text.split(" ")[1]:
                    lista_precio_usd.append(float(element.text.split(" ")[1].replace(",","")))
                    lista_precio_mxn.append(float(element.text.split(" ")[1].replace(",",""))*dolar)
                else:
                    lista_precio_usd.append(float(element.text.split(" ")[1]))
                    lista_precio_mxn.append(float(element.text.split(" ")[1])*dolar)
            elif element.text.split(" ")[0] == "MXN" and element.text != "":
                lista_precio_mxn.append(float(element.text.split(" ")[1].replace(",","")))
                lista_precio_usd.append(float(element.text.split(" ")[1].replace(",",""))/dolar)
            else:
                lista_precio_mxn.append(0)
                lista_precio_usd.append(0)
    except:
        print("Elemento tarifa no encontrado")
        lista_habitacion = ["-"]
    return lista_nombre,lista_precio_usd,lista_precio_mxn

def _fill_dataframe(df,date_consult,dateIN,dateOUT,nombres,precios_usd,precios_mxn,dolar):
    """Método utilizado para llenar el dataframe
       Parámetros: <df>           tipo: Pandas.DataFrame --> El dataframe instanciado 
                   <date_consult> tipo: Date             --> La fecha en que se realizó esta extracción
                   <dateIN>       tipo: str              --> La fecha de llegada
                   <dateOUT>      tipo: str              --> La fecha de salida
                   <nombres>      tipo: list             --> Los nombres de las habitaciones
                   <precios>      tipo: list             --> Los precios de las habitaciones
                   <dolar>        tipo: float            --> El precio del dólar al día
       Retorno: <df> tipo: Pandas:DataFrame --> El dataframe lleno con estos n resultados
       """
    #dateOUT = dateIN + datetime.timedelta(days=1)
    #dateIN = str(dateIN.year)+'/'+str(dateIN.month)+'/'+str(dateIN.day)
    #dateOUT = str(dateOUT.year)+'/'+str(dateOUT.month)+'/'+str(dateOUT.day)
    if len(nombres) != 0:
        for indx in range(len(nombres)):
            dicc_data = {"Hotel":"Iberostar Cancún",
                     "Fecha_consulta":str(consult_Date),
                     "Fecha_reserva_checkin":str(dateIN),
                     "Fecha_reserva_checkout":str(dateOUT),
                     "Nombre_habitacion":nombres[indx],
                     "Precio_habitacion_MXN":precios_mxn[indx],
                     "Precio_habitacion_USD":precios_usd[indx],
                     "Precio_dolar_al_dia":dolar}
            df = df.append(dicc_data,ignore_index = True)
    else: 
        dicc_data = {"Hotel":"Iberostar Cancún",
                     "Fecha_consulta":str(consult_Date),
                     "Fecha_reserva_checkin":str(dateIN),
                     "Fecha_reserva_checkout":str(dateOUT),
                     "Nombre_habitacion":"-",
                     "Precio_habitacion_MXN":0,
                     "Precio_habitacion_USD":0,
                     "Precio_dolar_al_dia":dolar}
        df = df.append(dicc_data,ignore_index = True)
    return df

def _wait(num):
    """Método utilizado para realizar una espera implicita
       Parámetros: <num> tipo: int --> Requiere de un número para indicar una espera
       Retorno: <None>"""
    time.sleep(num)

def _quit(driver):
    """Método utilizado para cerrar el driver
       Parámetros: <selenium.webdriver> tipo: selenium.webdriver --> Es el bot ya levantado y configurado
       Retorno: <None>"""
    driver.quit()
    return None

<div class="alert alert-block alert-warning">
    <h3><center>Implementación</center></h3>
</div>

In [15]:
###
# Obtención del precio del dólar
###
c = CurrencyRates()
mxn = c.get_rate('USD', 'MXN')
mxn

24.2688102894

In [16]:
###
# Ejecución de la información 
###
num_days = int(input("Ingresa el número de días a consultar: "))
dicc_ibero = {"Hotel":[],
             "Fecha_consulta":[],
             "Fecha_reserva_checkin":[],
             "Fecha_reserva_checkout":[],
             "Nombre_habitacion":[],
             "Precio_habitacion_MXN":[],
             "Precio_habitacion_USD":[],
             "Precio_dolar_al_dia":[]}
ibero_data = pd.DataFrame(dicc_ibero)
consult_Date = date.today()
today = datetime.date.today()
tomorrow = today + datetime.timedelta(days=1)
driver = _connect_iberostar()
_wait(5)
while num_days > 0:
    _update_page(driver,today,tomorrow)
    l1,l2,l3 = _get_price(driver,25.34)
    print(l1,l2,l3)
    ibero_data = _fill_dataframe(ibero_data,consult_Date,today,tomorrow,l1,l2,l3,str(25.34))
    today = tomorrow
    tomorrow = today + datetime.timedelta(days=1)
    num_days -= 1
_quit(driver)

Ingresa el número de días a consultar: 90
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []


In [17]:
###
# Limpieza de las habitaciones de nuestro interes
###
ibero_data = ibero_data.loc[(ibero_data["Nombre_habitacion"]=="Doble") |\
                            (ibero_data["Nombre_habitacion"]=="Doble Vista Mar") |\
                            (ibero_data["Nombre_habitacion"]=="Doble Vista Mar Lateral") |\
                            (ibero_data["Nombre_habitacion"]=="Doble Vista Mar Superior"),:]

In [18]:
###
# Salvando los archivos
###
ibero_data.to_csv("./Iberostar_data/Ibero_"+str(consult_Date)+"_comma.csv",quoting=csv.QUOTE_ALL,index=False,encoding="utf-8-sig")
ibero_data.to_csv("./Iberostar_data/Ibero_"+str(consult_Date)+"_pipe.csv",sep='|',quoting=csv.QUOTE_ALL,index=False,encoding="utf-8-sig")

In [ ]:
if lista_l[0].text.split(" ")[0] == 'USD':
    lista_precios_usd = list(map(lambda x:float(x.text.split(" ")[1].replace(",","")) if "," is x.text.split("-")[1] else float(x.text.split(" ")[1]),lista_l))
else:
    lista_precios_mxn = list(map(lambda x:float(x.get_attribute('price'))*25.34 if x.get_attribute('price')!="" else "-",lista_l))